In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install autogluon==1.0 --use-deprecated=legacy-resolver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.1/229.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.0/306.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.7/416.7 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 107.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

  Using cached torchvision-0.15.2-cp310-cp310-manylinux1_x86_64.whl (6.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 9.5 MB/s eta 0:00:00
  Using cached torch-2.0.1-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
  Using cached pillow-10.3.0-cp310-cp310-manylinux_2_28_x86_64.whl (4.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 20.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cuda_cupti_cu11-11.7.101-py3-none-manylinux1_x86_64.whl (11.8 MB)
  Using cached nvidia_cudnn_

In [ ]:
!pip install torchaudio==2.0.2 --force-reinstall

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import pywt
#from autogluon.timeseries import TimeSeriesPredictor
import pickle
import os
from numpy.fft import *
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import STL

In [ ]:
def preprocess_and_save(denoise = False, to_transform = [], threshold = 1e3):

    first_dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Gitex/Rabat_without_vapor.csv")
    second_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Gitex/merged_pwv.csv').drop(
    columns = ['report_id', 'station_name', 'city', 'organisation_name', 'latitude', 'longitude', 'sensor_altitude', 'height_of_station_above_sea_level', 'start_date',
               'total_column_water_vapour', 'total_column_water_vapour_combined_uncertainty', 'total_column_water_vapour_era5'])
    second_dataset['timestamp'] = pd.to_datetime(second_dataset['report_timestamp'])
    second_dataset.drop(columns = ['report_timestamp'], inplace = True)
    # Resample the hourly dataset to daily frequency
    daily_dataset = second_dataset.set_index('timestamp').resample('D').agg(['max', 'min', 'mean'])
    daily_dataset.index = daily_dataset.index.tz_localize(None)
    # Flatten the multi-level column index into a single level
    daily_dataset.columns = ['_'.join(col).strip() for col in daily_dataset.columns.values]

    # Concatenate daily max, min, mean values with the first dataset
    df = pd.concat([first_dataset.set_index('timestamp'), daily_dataset], axis=1)

    # Reset index to make 'timestamp' a regular column
    df.reset_index(inplace=True)

    df['datetime'] = pd.to_datetime(df['datetime'])
    df.drop_duplicates(subset="datetime", inplace=True)

    df.rename(columns={'name': 'item_id', "datetime" : "timestamp"}, inplace=True)

    irrelevant_columns = ['stations', 'description', 'severerisk', 'preciptype', 'sunrise', 'sunset', 'conditions']
    df.drop(columns=irrelevant_columns, inplace=True)


    correlated_cols = ["feelslikemax", "feelslikemin", "feelslike", "solarenergy", "uvindex"]
    df.drop(columns=correlated_cols, inplace=True)

    columns = df.columns.tolist()
    columns.remove('item_id')
    columns.insert(0, 'item_id')
    df = df[columns]

    numerical_columns = df.select_dtypes(include=[np.number]).columns

    for column in numerical_columns:
      if not df[column].isnull().any():
        continue
      # Apply STL decomposition
      stl = STL(df[column].interpolate(), period=365, robust=True)
      res = stl.fit()

      # Extract the seasonal and trend components
      seasonal_component = res.seasonal
      trend_component = res.trend

      # Create the deseasonalized series
      df_deseasonalized = df[column] - seasonal_component - trend_component

      # Interpolate missing values in the deseasonalized series
      df_deseasonalized_imputed = df_deseasonalized.interpolate(method="linear")

      # Add the seasonal component back to create the final imputed series
      df_imputed = df_deseasonalized_imputed + seasonal_component + trend_component

      # Update the original dataframe with the imputed values
      df[column] = df_imputed


    imputer2 = SimpleImputer(strategy="most_frequent")
    df[['icon']] = imputer2.fit_transform(df[['icon']])
    df['item_id'] = "Rabat"


    # Separate the timestamp and item_id columns
    timestamp_column = df['timestamp']
    item_id_column = df['item_id']
    icon_column = df["icon"]
    # Select the remaining columns (excluding timestamp and item_id)
    columns_to_scale = df.drop(columns=['timestamp', 'item_id', 'icon'])
    # Create and fit the StandardScaler
    scaler = StandardScaler()
    scaler.fit(columns_to_scale)
    # Transform the data using the fitted scaler
    scaled_data = scaler.transform(columns_to_scale)
    # Create a DataFrame with the scaled data and the original column names (excluding timestamp and item_id)
    scaled_df = pd.DataFrame(scaled_data, columns=columns_to_scale.columns)
    # Combine the timestamp and item_id columns with the scaled data
    df = pd.concat([item_id_column, timestamp_column, scaled_df, icon_column], axis=1)

    df = pd.get_dummies(df, columns=['icon'])
    df.drop(columns=['icon_rain'], inplace=True)

    # Define the file path where you want to save the scaler
    scaler_file_path = '/content/drive/MyDrive/Colab Notebooks/Gitex/PreprocessedData/scaler.pkl'
    # Save the scaler object to the file
    with open(scaler_file_path, 'wb') as file:
        pickle.dump(scaler, file)

    denoised_signals = {}

    if denoise:
        for col in to_transform:
            signal = df[col].values
            length = len(signal)
            fourier = rfft(signal, n=length)
            frequencies = rfftfreq(length, d=20e-3/length)
            fourier[frequencies > threshold] = 0
            denoised_signals[col] = irfft(fourier, n=length)
        denoised_df = pd.DataFrame(denoised_signals)
        # Update the original DataFrame by replacing the specified columns with the denoised data
        df.update(denoised_df)


    df.to_csv("/content/drive/MyDrive/Colab Notebooks/Gitex/PreprocessedData/data.csv", index=False)
    return df

In [ ]:
def train_and_eval(df, prediction_length = 30):
  train_data = df.iloc[:len(df)-prediction_length]
  test_data = df.iloc[len(df)-prediction_length:]

  predictor = TimeSeriesPredictor(target='precip',
                                  prediction_length=prediction_length,
                                  freq='D')
  predictor.fit(train_data, presets="best_quality", num_val_windows=3)

  predictions = predictor.predict(train_data)

  plt.figure(figsize=(20, 3))

  y_past = train_data["precip"]
  y_test = test_data["precip"]
  y_prep = predictions.loc["Rabat"]
  plt.plot(train_data['timestamp'], y_past, label="Past time series values")
  plt.plot(y_prep.index, y_prep["mean"], label="Mean forecast")
  plt.plot(y_prep.index, y_test, label="Future time series values")

  plt.fill_between(
    y_prep.index, y_prep["0.1"], y_prep["0.9"], color="red", alpha=0.1, label=f"10%-90% confidence interval"
  )

  # Zoom in on the last part of the graph
  zoom_index = int(0.99 * len(train_data))  # Zoom in on the last 10% of the forecast
  plt.xlim(train_data.loc[zoom_index, 'timestamp'], y_prep.index[-1])  # Set x-axis limits to the last 10% of the forecast

  plt.legend()
  plt.show()

In [ ]:
preprocess_and_save(df.copy(), denoise=True,
    to_transform=["tempmin", "tempmax", "temp", "dew", "humidity", "winddir", "sealevelpressure", "solarradiation", "cloudcover"],
    threshold=1e4+500)
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Gitex/PreprocessedData/data.csv")
train_and_eval(df)